In [ ]:
#################################################
# Imports and function definitions
#################################################
import tensorflow as tf
import tensorflow_hub as hub

# For saving 'feature vectors' into a txt file
import numpy as np

# Time for measuring the process time
import time

# Glob for reading file names in a folder
import glob
import os.path

In [ ]:
#################################################
# This function:
# Loads the JPEG image at the given path
# Decodes the JPEG image to a uint8 W X H X 3 tensor
# Resizes the image to 224 x 224 x 3 tensor
# Returns the pre processed image as 224 x 224 x 3 tensor
#################################################
def load_img(path):

  # Reads the image file and returns data type of string
  img = tf.io.read_file(path)

  # Decodes the image to W x H x 3 shape tensor with type of uint8
  img = tf.io.decode_jpeg(img, channels=3)

  # Resize the image to 224 x 244 x 3 shape tensor
  img = tf.image.resize_with_pad(img, 224, 224)

  # Converts the data type of uint8 to float32 by adding a new axis
  # This makes the img 1 x 224 x 224 x 3 tensor with the data type of float32
  # This is required for the mobilenet model we are using
  img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  return img

In [ ]:
def get_image_feature_vectors():

  i = 0

  start_time = time.time()

  print("---------------------------------")
  print ("Step.1 of 2 - mobilenet_v2_140_224 - Loading Started at %s" %time.ctime())
  print("---------------------------------")

  # Definition of module with using tfhub.dev handle
  module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4" 
  
  # Load the module
  module = hub.load(module_handle)

  print("---------------------------------")
  print ("Step.1 of 2 - mobilenet_v2_140_224 - Loading Completed at %s" %time.ctime())
  print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))

  print("---------------------------------")
  print ("Step.2 of 2 - Generating Feature Vectors -  Started at %s" %time.ctime())
 

  # Loops through all images in a local folder
  for filename in glob.glob('/content/sample_data/content/Cattle1/*.jpg'): #assuming gif
    i = i + 1

    print("-----------------------------------------------------------------------------------------")
    print("Image count                     :%s" %i)
    print("Image in process is             :%s" %filename)

    # Loads and pre-process the image
    img = load_img(filename)

    # Calculate the image feature vector of the img
    features = module(img)   
  
    # Remove single-dimensional entries from the 'features' array
    feature_set = np.squeeze(features)  

    # Saves the image feature vectors into a file for later use

    outfile_name = os.path.basename(filename).split('.')[0] + ".npz"
    out_path = os.path.join('/content/sample_data/content/Cattle1/feature-vectors/', outfile_name)

    # Saves the 'feature_set' to a text file
    np.savetxt(out_path, feature_set, delimiter=',')

    print("Image feature vector saved to   :%s" %out_path)
  
  print("---------------------------------")
  print ("Step.2 of 2 - Generating Feature Vectors - Completed at %s" %time.ctime())
  print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))
  print("--- %s images processed ---------" %i)

In [ ]:
get_image_feature_vectors()

---------------------------------
Step.1 of 2 - mobilenet_v2_140_224 - Loading Started at Sun Jul 25 15:45:51 2021
---------------------------------
---------------------------------
Step.1 of 2 - mobilenet_v2_140_224 - Loading Completed at Sun Jul 25 15:45:53 2021
--- 0.03 minutes passed ---------
---------------------------------
Step.2 of 2 - Generating Feature Vectors -  Started at Sun Jul 25 15:45:53 2021
-----------------------------------------------------------------------------------------
Image count                     :1
Image in process is             :/content/sample_data/content/Cattle1/Cattle1_2.jpg
Image feature vector saved to   :/content/sample_data/content/Cattle1/feature-vectors/Cattle1_2.npz
-----------------------------------------------------------------------------------------
Image count                     :2
Image in process is             :/content/sample_data/content/Cattle1/test1.jpg
Image feature vector saved to   :/content/sample_data/content/Cattle1/fe

In [ ]:
!pip install annoy

In [ ]:
# Annoy and Scipy for similarity calculation
from annoy import AnnoyIndex
from scipy import spatial
# json for storing data in json file
import json

In [ ]:
#################################################
# This function reads from 'image_data.json' file
# Looks for a specific 'filename' value
# Returns the product id when product image names are matched 
# So it is used to find product id based on the product image name
#################################################
def match_id(filename):
  with open('/content/sample_data/content/cattle_names.json') as json_file:

        seen = json.load(json_file)

        for line in seen:
          
          if filename==line['imageName']:
            print(line)
            return line['productId']
            break

In [ ]:
match_id("Cattle1_2")

{'imageName': 'Cattle1_2', 'productId': 'Cattle1_2'}


'Cattle1_2'

In [ ]:
#################################################
# This function; 
# Reads all image feature vectores stored in /feature-vectors/*.npz
# Adds them all in Annoy Index
# Builds ANNOY index
# Calculates the nearest neighbors and image similarity metrics
# Stores image similarity scores with productID in a json file
#################################################
def cluster():

  start_time = time.time()
  
  print("---------------------------------")
  print ("Step.1 - ANNOY index generation - Started at %s" %time.ctime())
  print("---------------------------------")

  # Defining data structures as empty dict
  file_index_to_file_name = {}
  file_index_to_file_vector = {}
  file_index_to_product_id = {}

  # Configuring annoy parameters
  dims = 1792
  n_nearest_neighbors = 100
  trees = 10000

  # Reads all file names which stores feature vectors 
  allfiles = glob.glob('/content/sample_data/content/Cattle1/feature-vectors/*.npz')

  t = AnnoyIndex(dims, metric='angular')

  for file_index, i in enumerate(allfiles):
    
    # Reads feature vectors and assigns them into the file_vector 
    file_vector = np.loadtxt(i)

    # Assigns file_name, feature_vectors and corresponding product_id
    file_name = os.path.basename(i).split('.')[0]
    file_index_to_file_name[file_index] = file_name
    file_index_to_file_vector[file_index] = file_vector
    file_index_to_product_id[file_index] = match_id(file_name)

    # Adds image feature vectors into annoy index   
    t.add_item(file_index, file_vector)

    print("---------------------------------")
    print("Annoy index     : %s" %file_index)
    print("Image file name : %s" %file_name)
    print("Product id      : %s" %file_index_to_product_id[file_index])
    print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))


  # Builds annoy index
  t.build(trees)

  print ("Step.1 - ANNOY index generation - Finished")
  print ("Step.2 - Similarity score calculation - Started ") 
  
  global named_nearest_neighbors

  # Loops through all indexed items
  for i in file_index_to_file_name.keys():

    # Assigns master file_name, image feature vectors and product id values
    master_file_name = file_index_to_file_name[i]
    master_vector = file_index_to_file_vector[i]
    master_product_id = file_index_to_product_id[i]

    # Calculates the nearest neighbors of the master item
    nearest_neighbors = t.get_nns_by_item(i, n_nearest_neighbors)

    # Loops through the nearest neighbors of the master item
    for j in nearest_neighbors:

      print(j)

      # Assigns file_name, image feature vectors and product id values of the similar item
      neighbor_file_name = file_index_to_file_name[j]
      neighbor_file_vector = file_index_to_file_vector[j]
      neighbor_product_id = file_index_to_product_id[j]

      # Calculates the similarity score of the similar item
      similarity = 1 - spatial.distance.cosine(master_vector, neighbor_file_vector)
      rounded_similarity = int((similarity * 10000)) / 10000.0

      # Appends master product id with the similarity score 
      # and the product id of the similar items
      named_nearest_neighbors.append({
        'similarity': rounded_similarity,
        'master_pi': master_product_id,
        'similar_pi': neighbor_product_id
        })

    print("---------------------------------") 
    print("Similarity index       : %s" %i)
    print("Master Image file name : %s" %file_index_to_file_name[i]) 
    print("Nearest Neighbors.     : %s" %nearest_neighbors) 
    print("--- %.2f minutes passed ---------" % ((time.time() - start_time)/60))

  
  print ("Step.2 - Similarity score calculation - Finished ") 

  # Writes the 'named_nearest_neighbors' to a json file
  with open('nearest_neighbors.json', 'w') as out:
    json.dump(named_nearest_neighbors, out)

  print ("Step.3 - Data stored in 'nearest_neighbors.json' file ") 
  print("--- Prosess completed in %.2f minutes ---------" % ((time.time() - start_time)/60))

In [ ]:
named_nearest_neighbors = []
cluster()

---------------------------------
Step.1 - ANNOY index generation - Started at Sun Jul 25 16:24:59 2021
---------------------------------
{'imageName': 'test4', 'productId': 'test4'}
---------------------------------
Annoy index     : 0
Image file name : test4
Product id      : test4
--- 0.00 minutes passed ---------
{'imageName': 'Cattle2_3', 'productId': 'Cattle2_3'}
---------------------------------
Annoy index     : 1
Image file name : Cattle2_3
Product id      : Cattle2_3
--- 0.00 minutes passed ---------
{'imageName': 'test3', 'productId': 'test3'}
---------------------------------
Annoy index     : 2
Image file name : test3
Product id      : test3
--- 0.00 minutes passed ---------
{'imageName': 'Cattle1_3', 'productId': 'Cattle1_3'}
---------------------------------
Annoy index     : 3
Image file name : Cattle1_3
Product id      : Cattle1_3
--- 0.00 minutes passed ---------
{'imageName': 'test2', 'productId': 'test2'}
---------------------------------
Annoy index     : 4
Image f

In [ ]:
import pandas as pd
df = pd.DataFrame(named_nearest_neighbors)

In [ ]:
results = df.head(400).sort_values(by = 'similarity', ascending = False)
results.loc[results['similarity'] <= 0.80000, 'Match/No Match'] = 'No Match'
results.loc[results['similarity'] > 0.80000, 'Match/No Match'] = 'Match'
results.head(100)
results.to_csv('/content/sample_data/content/Cattle1/results.csv')